In [ ]:
import torchaudio
from collections import defaultdict
from speechbrain.inference.speaker import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")

/home/idio/anaconda3/lib/python3.12/site-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="",
    api_key="",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='celebrity_voices')]


In [ ]:
def search_sim(usr_path):
    signal, fs = torchaudio.load(usr_path)
    user_emb = classifier.encode_batch(signal).flatten()
    search_results = qdrant_client.search(
        collection_name="celebrity_voices",
        query_vector=user_emb,
        limit=100
    )
    speaker_best = defaultdict(lambda: {"score": -float("inf"), "payload": None})

    for res in search_results:
        speaker_id = res.payload["id"]
        if res.score > speaker_best[speaker_id]["score"]:
            speaker_best[speaker_id] = {"score": res.score, "payload": res.payload}

    # Sort speakers by their best score
    top_speakers = sorted(speaker_best.values(), key=lambda x: x["score"], reverse=True)

    return top_speakers[:5]

In [33]:
top_speakers = search_sim('ernest.wav')

/tmp/ipykernel_14091/420506408.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = qdrant_client.search(


In [34]:
top_speakers

[{'score': 0.2734542,
  'payload': {'id': 'id10297', 'path': 'o1EKUvjvNHY', 'num': '00009'}},
 {'score': 0.2673011,
  'payload': {'id': 'id10278', 'path': 'DSGE2F5sTcg', 'num': '00001'}},
 {'score': 0.23668143,
  'payload': {'id': 'id10309', 'path': 'rqaAm4bEsXc', 'num': '00004'}},
 {'score': 0.22416486,
  'payload': {'id': 'id10304', 'path': 'HTL8iLI75TY', 'num': '00001'}},
 {'score': 0.22125077,
  'payload': {'id': 'id10300', 'path': '1ZyvrJaiLQk', 'num': '00031'}}]

In [14]:
import pandas as pd

df = pd.read_csv('dataset/vox1_meta.csv',delimiter='\t')

In [35]:
for res in top_speakers:
    name = df[df['VoxCeleb1 ID'] == res['payload']['id']]['VGGFace1 ID'].values[0]
    name = name.replace('_', ' ')
    print('Name:', name, '| Score:', res['score'] )

Name: Eric Roberts | Score: 0.2734542
Name: Edward Asner | Score: 0.2673011
Name: Ezra Miller | Score: 0.23668143
Name: Eugene Levy | Score: 0.22416486
Name: Ernest Borgnine | Score: 0.22125077


In [36]:
from speechbrain.inference.speaker import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")

In [40]:
score, prediction = verification.verify_files("ernest.wav", "user_audio.wav")

In [41]:
score

tensor([0.0193])

In [42]:
prediction

tensor([False])